# Polyphonic Music 
Below are the source files (MIDI) for the 4 datasets evaluated in the paper (split in train, validation and test sets). You can generate piano-rolls from the source files by transposing each sequence in C major or C minor and sampling frames every eighth note (quarter note for JSB chorales) following the beat information present in the MIDI file. Alternatively, pickled piano-rolls for use with the Python language are also provided

Dataset [here](http://www-etud.iro.umontreal.ca/~boulanni/icml2012)

This will load a dictionary with 'train', 'valid' and 'test' keys, with the corresponding values being a list of sequences. Each sequence is itself a list of time steps, and each time step is a list of the non-zero elements in the piano-roll at this instant (in MIDI note numbers, between 21 and 108 inclusive). 


MIDI notes are numbered from 0 to 127 assigned to C-1 to G9. This corresponds to a range of 8.175799 to 12543.85 Hz (assuming equal temperament and 440 Hz A4) and extends beyond the 88 note piano range from A0 to C8.

In [ ]:
import os
os.getcwd()

In [ ]:
from utils import *

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from scipy.special import logsumexp
from torchsummary import summary
import cmath

In [ ]:
path = 'C:\\Users\\kmorales\\Google Drive\\PhD\\Music_data' 

path = path + '\\jsb_raw_batchsize16_seqlen100.npz'
data_set = np.load(path)

print(data_set.files)
print(data_set['x_train'].shape)
print(data_set['x_valid'].shape)
print(data_set['x_test'].shape)

In [ ]:
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"  

In [ ]:
def analysis(train,test,batch):
    train_loader = torch.utils.data.DataLoader(train,batch_size=batch,shuffle=False)
    test_loader = torch.utils.data.DataLoader(test,batch_size=batch,shuffle=False)
    return train_loader, test_loader

In [ ]:
batch_sz = 16
train_loader, test_loader = analysis(data_set['x_train'], data_set['x_valid'], batch_sz)
test_loader_elbo =  torch.utils.data.DataLoader(data_set['x_test'],batch_size=1,shuffle=False)

In [ ]:
for batch_idx, data in enumerate(train_loader):
    data = data.float()
    data = data.transpose(0, 1)
    print(data.shape)

# Models

In [ ]:
#from vrnn_mnist_ import VRNN
from vrnn2_mnist_ import VRNN
from pcm2_mnist_ import PCM2
from hmm2_mnist_ import HMM
from pcm_c_mnist import PCM_c
from pcm_c2_mnist import PCM_c2
#from pcm_c3_mnist import PCM_c3

# VRNN

https://github.com/clinicalml/structuredinference/blob/master/parse_args_dkf.py




In [ ]:
# Change the directory

folder = 'jsb/' # muse, notting , piano 

#--------------------------------------------
img = True
if img:
    x_dim = 88
else:
    x_dim = 1

h_dim_vrnn = 300

h_dim = h_dim_vrnn


z_dim = 300

px_dim = 100

pz_dim = 100

qz_dim = 500 



n_layers =  1

n_epochs = 200

n_samples = 100

clip = 10

learning_rate = 0.003#0.001


print_every = 100

save_every = 2

#--------------------------------------------
#Intializing empty arrays to store train and test NLL
#manual seed
seed = 123
torch.manual_seed(seed)
#--------------------------------------------
# Model
model = VRNN(x_dim, h_dim, z_dim, px_dim, pz_dim, qz_dim, n_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-4)
#--------------------------------------------

model_name = 'vrnn_state_'
path_save = folder+model_name





In [ ]:
summary(model)

In [ ]:
train_LOSS_1, test_LOSS_1= run_model(train_loader,test_loader,model,optimizer,batch_sz,clip, path_save, n_epochs ,print_every, img, save_every)

In [ ]:

epoch_model = 199
# learning_rate = 0.005
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-4)


path = path_save+str(epoch_model)+'.pth'
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)    



In [ ]:
# train_LOSS_1, test_LOSS_1,_,_,_,_= run_model(train_loader,test_loader,model,optimizer,batch_sz,clip, path_save, n_epochs ,train_NLL, train_KLD, train_LOSS, test_NLL, test_KLD, test_LOSS,print_every, img, save_every)

In [ ]:

elb, lpx = ELBO(test_loader_elbo, model2, n_samples,folder, model_name)
print(np.mean(elb))
print(np.mean(lpx))

# PCM

In [ ]:
name_model = 'PCM'
number_parameters(name_model, h_dim_vrnn, pz_dim, px_dim, qz_dim ,x_dim, z_dim)


In [ ]:
# H dim
h_dim = 84

# # Model
#--------------------------------------------

model2 = PCM2(x_dim, h_dim, z_dim,px_dim, pz_dim, qz_dim, n_layers)

optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate, weight_decay= 1e-4)
#--------------------------------------------

model_name = 'pcm_state_'
path_save = folder+model_name
#--------------------------------------------


In [ ]:
summary(model2)

In [ ]:
train_LOSS_2, test_LOSS_2= run_model(train_loader,test_loader,model2,optimizer2,batch_sz,clip, path_save, n_epochs,print_every, img, save_every)

In [ ]:

epoch_model = 199
# learning_rate = 0.005
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-4)


path = path_save+str(epoch_model)+'.pth'
checkpoint = torch.load(path)
model2.load_state_dict(checkpoint['model_state_dict'])
optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)    



In [ ]:
elb2, lpx2 = ELBO(test_loader_elbo, model2, n_samples,folder, model_name)

In [ ]:
print(np.mean(elb2))
print(np.mean(lpx2))

# PCM_c

In [ ]:
name_model = 'PCM_c'
number_parameters(name_model, h_dim_vrnn, pz_dim, px_dim, qz_dim ,x_dim, z_dim)

In [ ]:
# H dim

h_dim = 71

# Model
#--------------------------------------------

model3 = PCM_c(x_dim, h_dim, z_dim,px_dim, pz_dim, qz_dim, n_layers)

optimizer3 = torch.optim.Adam(model3.parameters(), lr=learning_rate, weight_decay= 1e-4)
#--------------------------------------------
model_name = 'pcm_c_state_'
path_save = folder+model_name
#--------------------------------------------


In [ ]:
summary(model3)

In [ ]:
train_LOSS_3, test_LOSS_3 = run_model(train_loader,test_loader,model3,optimizer3,batch_sz,clip, path_save, n_epochs ,print_every, img, save_every)

In [ ]:
epoch_model = 199
# learning_rate = 0.005
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-4)


path = path_save+str(epoch_model)+'.pth'
checkpoint = torch.load(path)
model3.load_state_dict(checkpoint['model_state_dict'])
optimizer3.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)

In [ ]:
elb3, lpx3 = ELBO(test_loader_elbo, model3, n_samples,folder, model_name)

In [ ]:
print(np.mean(elb4))
print(np.mean(lpx4))

# PCM_c2

In [ ]:
name_model = 'PMC_c2'
number_parameters(name_model, h_dim_vrnn, pz_dim, px_dim, qz_dim ,x_dim, z_dim)


In [ ]:
# H dim

h_dim = 71

# Model
#--------------------------------------------
model4 = PCM_c2(x_dim, h_dim, z_dim,px_dim, pz_dim, qz_dim, n_layers)

optimizer4 = torch.optim.Adam(model4.parameters(), lr=learning_rate, weight_decay= 1e-4)
#--------------------------------------------
model_name = 'pcm_c2_state_'
path_save = folder+model_name
#--------------------------------------------


In [ ]:
summary(model4)

In [ ]:
train_LOSS_4, test_LOSS_4 = run_model(train_loader,test_loader,model4,optimizer4,batch_sz,clip, path_save, n_epochs, print_every, img, save_every)

In [ ]:
epoch_model = 199
# learning_rate = 0.005
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-4)


path = path_save+str(epoch_model)+'.pth'
checkpoint = torch.load(path)
model4.load_state_dict(checkpoint['model_state_dict'])
optimizer4.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
elb4, lpx4 = ELBO(test_loader_elbo, model4, n_samples,folder, model_name)

In [ ]:
print(np.mean(elb5))
print(np.mean(lpx5))